In [20]:
from multiprocessing import Process, Pipe
import signal
import logging
import random
import numpy as np

In [21]:
filename = 'brancha.txt'
with open(filename, 'w+') as f:
    f.write('')
f.close()
msg_filename = 'brancha_msg.txt'
with open(msg_filename, 'w+') as f:
    f.write('')
f.close()

## Logging

* `logger.setLevel(logging.DEBUG)` to debug
* `logger.setLevel(logging.WARNING)` to only log the final result (deadlocked or not deadlocked)

In [22]:
logging.basicConfig()
logger = logging.getLogger('MPLogger')
#logger.setLevel(logging.DEBUG)
logger.setLevel(logging.WARNING)

## Set parameters

In [24]:
num_processes = 5
wfg = non_deadlocked_wfg
initiator = 2
timeout = 1
msg_gaurentee = 0.99
num_experiments = 100 # Number of times to simulate

## Timeout

* In diffusion based methods, if initiator receives messages for all processes in its dependent set, it is deadlocked.
* Else if it did not receive all messages before timeout, it will assume that it is not deadlocked
* Noisy channel with less than 100% delivery gaurentee can only mistake deadlocked situation as not deadlocked, vice versa cannot happen

In [25]:
class Timeout:
    def __init__(self, seconds=1, error_message='Timeout'):
        self.seconds = seconds
        self.error_message = error_message
    def handle_timeout(self, signum, frame):
        raise TimeoutError(self.error_message)
    def __enter__(self):
        signal.signal(signal.SIGALRM, self.handle_timeout)
        signal.alarm(self.seconds)
    def __exit__(self, type, value, traceback):
        signal.alarm(0)

## Message passing gaurentees

* if prob == 1, then gaurentee is 100%

In [26]:
def noisy_send(pipe, msg):
    send_prob = np.random.uniform(0, 1)
    if send_prob <= msg_gaurentee:
        pipe.send(msg)
        logger.debug("Succesfully sent: {}".format(msg) + " Prob generated: {}".format(send_prob))
        with open(msg_filename, 'a+') as f: f.write('1')
    else:
        logger.debug("Failed: {}".format(msg) + " Prob generated: {}".format(send_prob))

## Dependency lists

* Funcion to get dependency lists for each process

In [27]:
def get_waiting_on(id, wfg):
    waiting_on = []
    for pi, pj in wfg:
        if pj == id:
            waiting_on.append(pi)
    return waiting_on

In [28]:
def get_waited_by(id, wfg):
    waited_by = []
    for pi, pj in wfg:
        if pi == id:
            waited_by.append(pj)
    return waited_by

## Brancha-Toueg

In [29]:
def process(id, num_processes, send_pipes, recv_pipes, waiting_on, waited_by, initiator=False, timeout=10):
    np.random.seed(random.randint(0, 100000))
    
    notified, free = False, False
    n = len(waiting_on)
    num_granted = 0
    
    logger.debug("{} waiting_on {}".format(id, waiting_on))
    
    try:
        with Timeout(timeout):
            
            # =================================================================================
            """
            **Initiate a diffusion computation for a blocked process Pi :**

                * send query(i, i, j) to all processes Pj in the dependent set DSi of Pi

                `       
                numi (i):= |DSi |; 
                waiti (i):= true;
                `
            """
            
            if initiator:
                logger.debug('Initiator {}'.format(id))
                notified = True
                for pj in waiting_on: noisy_send(send_pipes[pj], 'notify')
                if n == 0:
                    free = True
                    for pj in waited_by: noisy_send(send_pipes[pj], 'grant')
                        
                    
            
            # =================================================================================

            while True:
                for recv_id, conn in enumerate(recv_pipes):
                    while conn.poll():
                        m = conn.recv()
                        
                        if m == 'notify' and not initiator:
                            notified = True
                            for pj in waiting_on: noisy_send(send_pipes[pj], 'notify')
                            if n == 0:
                                free = True
                                for pj in waited_by: noisy_send(send_pipes[pj], 'grant')
                                    
                        if m == 'grant':
                            n = 0
                            if not free:
                                for pj in waited_by: noisy_send(send_pipes[pj], 'grant')
                                free = True
                if initiator:
                    if n == 0 or free:
                        logger.error('Not Deadlocked')
                        with open(filename, 'a+') as f: f.write("{} {}\n".format(msg_gaurentee, 1))
                        return
                    
                if free: return
                    
                    
                                    
    except TimeoutError as e:
        if initiator:
            if n == 0 or free:
                logger.error('Not Deadlocked!')
                with open(filename, 'a+') as f: f.write("{} {}\n".format(msg_gaurentee, 1))
            else:
                logger.error('Deadlocked')
                with open(filename, 'a+') as f: f.write("{} {}\n".format(msg_gaurentee, 0))